In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re

In [2]:
base_url = "https://losangeles.craigslist.org/search/wst/apa"
usc_lat = 34.0224
usc_lon = -118.2851
search_distance = 3 
target_listings = 1000
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'}

print("analysis for", target_listings, "lisngs within 3 mi from usc")

analysis for 1000 lisngs within 3 mi from usc


In [3]:
all_listings= []
offset = 0
page_number = 1

while len(all_listings) < target_listings:
    params = {'search_distance': 3, 'postal': '90007', 's': offset}
    print("\nPage", page_number, "(offset", offset, "):")
    try:
        response = requests.get(base_url, params = params, headers = headers)
        if response.status_code != 200:
            print(response.status_code)
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        listings_on_page = soup.find_all('li', class_='cl-static-search-result')
        
        if len(listings_on_page) == 0:
            break
        print(len(listings_on_page), "litings")
        
        page_count = 0
        for listing in listings_on_page:
            #this scrapes the name 
            title_elem = listing.find('div', class_='title')
            if title_elem:
                name = title_elem.get_text().strip()
            else:
                name = ""
            
            # gets the prices
            price_elem = listing.find('div', class_='price')
            if price_elem:
                price = price_elem.get_text().strip()
            else:
                price = ""
            
            # sprapes locaton
            location_elem = listing.find('div', class_ = 'location')
            if location_elem:
                location = location_elem.get_text().strip()
            else:
                location = ""
            
            #num ofbeds
            housing_elem = listing.find('div', class_= 'meta')
            bedrooms = ""
            if name:
                name_lower = name.lower()
                #2br fornat 
                bed_match = re.search(r'(\d+)\s*br\b', name_lower)
                if bed_match:
                    bedrooms = bed_match.group(1) + "br"
                #2bed/2 bedroom format
                if not bedrooms:
                    bed_match = re.search(r'(\d+)\s*bed', name_lower)
                    if bed_match:
                        bedrooms = bed_match.group(1) + "br"
                #studio
                if not bedrooms and 'studio' in name_lower:
                    bedrooms = "studio"
                # studio/1B format
                if not bedrooms:
                    bed_match = re.search(r'(\d+)b\b', name_lower)
                    if bed_match:
                        bedrooms = bed_match.group(1) + "br"
                        
            #getting thhe url
            link_elem = listing.find('a', href=True)
            if link_elem:
                url = link_elem['href']
                if not url.startswith('http'):
                    url = 'https://losangeles.craigslist.org' + url
            else:
                url = ""
            
            #zip
            zip_code = ""
            if location:
                zip_match = re.search(r'\b(90\d{3})\b', location)
                if zip_match:
                    zip_code = zip_match.group(1)
            
            #only want useful lsitings
            if name or price:
                listing_data = {'name': name, 'address': location, 'zip_code': zip_code, 'price_raw': price, 'bedrooms_raw': bedrooms, 'url': url}                
                all_listings.append(listing_data)
                page_count = page_count + 1
        print(page_count, "listings w a listing ct. :", len(all_listings))
        if len(all_listings) >= target_listings:
            break
            
        offset = offset + 120
        page_number = page_number + 1
        
    except Exception as error:
        print(error)
        break
print("total listings :", len(all_listings))


Page 1 (offset 0 ):
316 litings
316 listings w a listing ct. : 316

Page 2 (offset 120 ):
316 litings
316 listings w a listing ct. : 632

Page 3 (offset 240 ):
316 litings
316 listings w a listing ct. : 948

Page 4 (offset 360 ):
316 litings
316 listings w a listing ct. : 1264
total listings : 1264


In [4]:
neighborhood_zips = {'university park': '90007', 'usc': '90007', 'u.s.c': '90007', 'u s c': '90007', 'university of southern california': '90089', 'koreatown': '90006', 'korea town': '90006', 'ktown': '90006', 'k-town': '90006', 'pico union': '90006', 'pico-union': '90006', 'pico': '90006', 'downtown': '90015', 'dtla': '90015', 'down town': '90015', 'fashion district': '90015', 'adams': '90018', 'west adams': '90018', 'w adams': '90018', 'jefferson park': '90018', 'mid city': '90019', 'mid-city': '90019', 'midcity': '90019', 'arlington heights': '90019', 'south la': '90037', 'south los angeles': '90037', 'south l.a.': '90037', 'exposition park': '90037', 'expo park': '90037', 'historic south central': '90037', 'westlake': '90057', 'west lake': '90057', 'macarthur park': '90057', 'mac arthur park': '90057', 'koreatown north': '90020', 'mid wilshire': '90005', 'mid-wilshire': '90005', 'wilshire': '90005', 'hancock park': '90004', 'windsor square': '90004', 'koreatown east': '90010', 'echo park': '90026', 'echopark': '90026', 'silver lake': '90026', 'silverlake': '90026', 'chinatown': '90012', 'china town': '90012', 'downtown la': '90013', 'historic core': '90014', 'financial district': '90017', 'arts district': '90021', 'crenshaw': '90008', 'baldwin hills': '90008', 'leimert park': '90008', 'view park': '90008'}
for listing in all_listings:
    if not listing['zip_code'] and listing['address']:
        address_lower = listing['address'].lower()
        for neighborhood, zip_code in neighborhood_zips.items():
            if neighborhood in address_lower:
                listing['zip_code'] = zip_code
                break

with_zip = sum(1 for l in all_listings if l['zip_code'])
print("Listings w zip :", with_zip, "from", len(all_listings))


Listings w zip : 476 from 1264


In [5]:

if len(all_listings) > 0:
    count_name = sum(1 for l in all_listings if l['name'])
    count_address = sum(1 for l in all_listings if l['address'])
    count_zip = sum(1 for l in all_listings if l['zip_code'])
    count_price = sum(1 for l in all_listings if l['price_raw'])
    count_beds = sum(1 for l in all_listings if l['bedrooms_raw'])
    zip_codes = list(set(l['zip_code'] for l in all_listings if l['zip_code']))
    
    print("num listinfs:", len(all_listings))
    print("has names:", count_name)
    print("with addresses:", count_address)
    print("haszip codes:", count_zip)
    print("has prices:", count_price)
    print("has bedrooms:", count_beds)
    print(len(zip_codes), "zipcodes:")
    print(sorted(zip_codes))
    
else:
    print("no listinfgs")

num listinfs: 1264
has names: 1264
with addresses: 1264
haszip codes: 476
has prices: 1264
has bedrooms: 696
11 zipcodes:
['90005', '90006', '90007', '90008', '90011', '90015', '90018', '90019', '90026', '90037', '90057']


In [6]:
output_file = '../data/raw/apartments_raw.json'
with open(output_file, 'w') as file:
    json.dump(all_listings, file, indent =2)

print(output_file)
print("num listings saved:", len(all_listings))

../data/raw/apartments_raw.json
num listings saved: 1264


#sources:
https://oxylabs.io/blog/scraping-real-estate-data